In [1]:
import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm
from dask import delayed
import dask.array as da

np.random.seed(13)

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [2]:
C = 2 # number of campaigns
U = 10 # number of customers.
H = 3 # number of channels.
D = 7 # number of planning days.
I = 3 # number of quota categories.
P = 10 # number of priority categories.

#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [3]:
e_cu = np.random.choice(2,(C, U))
#e_cu = np.ones((C, U), dtype='int8')
e_cu

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1, 0]])

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
q_ic = np.random.choice(2, (I,C))
#q_ic = np.zeros((I,C), dtype='int8')
q_ic

array([[1, 1],
       [0, 0],
       [0, 1]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [5]:
r_p = np.random.choice(100, P)
#r_p = np.ones(P, dtype='int8')
rp_c = np.array([r_p[r] for r in np.random.choice(P, C)])
rp_c

array([28, 18])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [6]:
b = 7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [7]:
k = 3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [8]:
l_c = np.random.choice([2,3,4],C)
l_c

array([4, 2])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [9]:
m_i = np.random.choice([4,3,5],I)
n_i = np.random.choice([1,3,2],I)
#m_i = np.ones((I), dtype='int8')*10
#n_i = np.ones((I), dtype='int8')*10

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [10]:
t_hd = np.random.choice([U*.7, U*.6, U*.5], (H, D))

# Model

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [11]:
X_cuhd = np.zeros((C,U,H,D), dtype='int')

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall h \in H,\forall d \in D
$$

In [12]:
def eligibility(X):
    return np.all([X_cuhd[:,:,h,d]<=e_cu for h in range(H) for d in range(D)])

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [13]:
def one_channel(X):
    return np.all(X.sum(axis=2)<=1)

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [14]:
def weekly_limitation(X):
    return np.all(X.sum(axis=(0,2,3))<=b)

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [15]:
def daily_limitation(X):
    return np.all(X.sum(axis=(0,2))<=k)

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [16]:
def campaign_limitation(X):
    return np.all(X.sum(axis=(2,3))<=l_c.reshape((C,1)))

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [17]:
def weekly_quota(X):
    return np.all([X_cuhd.sum(axis=(2,3)).T * q_ic[i, ].T  <= m_i[i] for i in range(I)])

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [18]:
def daily_quota(X):
    return np.all([np.all(X_cuhd.sum(2).T * q_ic[i,].T <= n_i[i]) for i in range(I)])

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [19]:
def channel_capacity(X):
    return np.all(X.sum(axis=(0,1))<=t_hd)

##### - check criterion

In [20]:
def check(X):
    if not eligibility(X):
        return False
    if not one_channel(X):
        return False
    if not daily_limitation(X):
        return False
    if not weekly_limitation(X):
        return False
    if not campaign_limitation(X):
        return False
    if not daily_quota(X):
        return False
    if not weekly_quota(X):
        return False
    if not channel_capacity(X):
        return False
    return True

In [21]:
%%time
#for d in trange(D, desc=f"Days Loop for campaign-{u}"):
#    for c in tqdm(np.argsort(-rp_c), desc="Campaigns Loop"):
#        for h in range(H):#trange(H, desc=f"Channels Loop at Day-{d}"):
#            for u in range(0,U,10):#trange(U, desc=f"Users Loop On Campaign-{c}"):
#                X_cuhd[c,u:(u+10),h,d]=1
#                if not check(X_cuhd):
#                    X_cuhd[c,u:(u+10),h,d]=0

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [22]:
def check(X):
    if not eligibility(X):
        return False
    if not one_channel(X):
        return False
    if not weekly_limitation(X):
        return False
    if not daily_limitation(X):
        return False
    if not campaign_limitation(X):
        return False
    if not weekly_quota(X):
        return False
    if not daily_quota(X):
        return False
    if not channel_capacity(X):
        return False
    return True

In [23]:
%%time
X_cuhd = np.zeros((C,U,H,D), dtype='int')
for c in tqdm(np.argsort(-rp_c), desc="Campaigns Loop"):
    for d in trange(D, desc=f"Days Loop for campaign-{c}"):
        for h in range(H):#trange(H, desc=f"Channels Loop at Day-{d}, Campapaign-{c}"):
            for u in range(U):#trange(U, desc=f"Users Loop On Campaign-{c}"):
                X_cuhd[c,u,h,d]=1
                if not check(X_cuhd):
                    X_cuhd[c,u,h,d]=0

Campaigns Loop:   0%|          | 0/2 [00:00<?, ?it/s]

Days Loop for campaign-0:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-1:   0%|          | 0/7 [00:00<?, ?it/s]

CPU times: user 255 ms, sys: 21.1 ms, total: 276 ms
Wall time: 312 ms


In [24]:
np.matmul(rp_c, X_cuhd.sum(axis=(1,2,3)))

192

- Sort Campaigns by rp_c
- Map every criteria to function

##kucuk sayilar ile algoritmayi test edelim, ve bb solution burada validate edelim.